In [1]:
library(nichenetr)
library(tidyverse)

── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [5]:
## We load the networks generated in the ParameterOptimization script
lr_network_Omnipath = readRDS("NNET_Custom/lig_rec_sources.rds")
sig_network_Omnipath = readRDS("NNET_Custom/sig_Network.rds")
gr_network_Omnipath = readRDS("NNET_Custom/gr_Network.rds")

# Construct NicheNet’s ligand-target model from unoptimized data source weights


In [6]:
## The interactions are weighted only based in the number of data sources that
## report them
All_sources <- unique(c(lr_network_Omnipath$source,
    sig_network_Omnipath$source, gr_network_Omnipath$source))

In [7]:
my_source_weights_df <- 
     tibble(source = All_sources, weight = rep(1,length(All_sources)))

In [8]:
weighted_networks <- construct_weighted_networks(
    lr_network = lr_network_Omnipath, 
    sig_network = sig_network_Omnipath, 
    gr_network = gr_network_Omnipath, 
    source_weights_df = my_source_weights_df)

In [9]:
weighted_networks

from,to,weight
<chr>,<chr>,<dbl>
_ADD1_ADD2_EPB41_GYPC_SLC2A1_SLC4A1_STOM,SPTB,1
_APH1A_NCSTN_PSEN1_PSENEN,APP,1
_APH1A_NCSTN_PSEN1_PSENEN,DSCAM,1
_APH1A_NCSTN_PSEN1_PSENEN,DSCAML1,1
_APH1A_NCSTN_PSEN1_PSENEN,NOTCH1,1
_APH1A_NCSTN_PSEN1_PSENEN,NOTCH2,1
_APH1A_NCSTN_PSEN1_PSENEN,NOTCH3,1
_APH1A_NCSTN_PSEN1_PSENEN,NOTCH4,1
_C2_C3_C4A_C4B_2,C5,1


In [10]:
## We read the results of the optimization 
resultsOptimization <- readRDS("NNET_Custom/Optimization_results.rds")

optimized_parameters = resultsOptimization %>% 
    process_mlrmbo_nichenet_optimization(
        source_names = my_source_weights_df$source %>% unique())

In [11]:
weighted_networks <- apply_hub_corrections(
    weighted_networks = weighted_networks, 
    lr_sig_hub = optimized_parameters$lr_sig_hub, 
    gr_hub = optimized_parameters$gr_hub)
saveRDS(weighted_networks,"NNET_Custom/weighted_networksNonSourceWeights.rds")

In [12]:
#We now generate the matrix containing the ligand-target regulatory potential scores based on the 
#weighted integrated networks.


ligands <- as.list(unique(lr_network_Omnipath$from))

ligand_target_matrix <- construct_ligand_target_matrix(
    weighted_networks = weighted_networks, 
    ligands = ligands, 
    algorithm = "PPR", 
    damping_factor = optimized_parameters$damping_factor, 
    ltf_cutoff = optimized_parameters$ltf_cutoff)
saveRDS(ligand_target_matrix,"NNET_Custom/ligand_target_matrixNoweights.rds")

Warning message in construct_ligand_tf_matrix(weighted_networks, ligands, ltf_cutoff, :
“One or more ligands of interest not present in the ligand-receptor network 'lr_network'. You can possibly ignore this warning if you provided your own ligand_receptor network to the weighted networks.”


In [13]:
## Here we also take into account the optimized source weights
weighted_networks <- construct_weighted_networks(
    lr_network = lr_network_Omnipath, 
    sig_network = sig_network_Omnipath, 
    gr_network = gr_network_Omnipath,
    source_weights_df = optimized_parameters$source_weight_df)

weighted_networks <- apply_hub_corrections(
    weighted_networks = weighted_networks, 
    lr_sig_hub = optimized_parameters$lr_sig_hub, 
    gr_hub = optimized_parameters$gr_hub)

ligand_target_matrix = construct_ligand_target_matrix(
    weighted_networks = weighted_networks, 
    ligands = ligands, 
    algorithm = "PPR", 
    damping_factor = optimized_parameters$damping_factor, 
    ltf_cutoff = optimized_parameters$ltf_cutoff)
saveRDS(ligand_target_matrix,"NNET_Custom/ligand_target_matrixWithweights.rds")
saveRDS(weighted_networks,"NNET_Custom/weighted_networksWithSourceWeights.rds")

Warning message in construct_ligand_tf_matrix(weighted_networks, ligands, ltf_cutoff, :
“One or more ligands of interest not present in the ligand-receptor network 'lr_network'. You can possibly ignore this warning if you provided your own ligand_receptor network to the weighted networks.”


[1] 1.781689

[1] 0.2316659

In [59]:
length(rownames(ligand_target_matrix))

[1] 13242

In [67]:
length(unique(gr_network_Omnipath$to))

[1] 9230

In [68]:
length(unique(sig_network_Omnipath$to))

[1] 6540

In [145]:
length(unique(c(s,l,sf,g,gf,lf)))

[1] 13242

In [97]:
length(unique(weighted_networks$lr_sig$to))

[1] 7119

In [108]:
length(s)

[1] 7119

In [105]:
s=unique(weighted_networks$lr_sig$to)
g=unique(weighted_networks$gr$to)
l=unique(lr_network_Omnipath$to)

In [144]:
s=unique(sig_network_Omnipath$to)
sf=unique(sig_network_Omnipath$from)
g=unique(gr_network_Omnipath$to)
l=unique(lr_network_Omnipath$to)
gf=unique(gr_network_Omnipath$from)
lf=unique(lr_network_Omnipath$from)

In [122]:
ligand_target_matrix["_ADD1_ADD2_EPB41_GYPC_SLC2A1_SLC4A1_STOM",]

CALM1          S100A10            WNT5A           CXCL16 
               0                0                0                0 
          CCL3L1             IL10           FLT3LG            CXCL9 
               0                0                0                0 
            CCL5             CCL8             CCL4             DKK1 
               0                0                0                0 
          CXCL10             TP53           TNFSF9          TNFSF10 
               0                0                0                0 
            FADD            GNRH1           CXCL11             IL13 
               0                0                0                0 
            BMP2             GDF5              NDP              IL3 
               0                0                0                0 
           IL12A              SHH              IHH             WNT1 
               0                0                0                0 
           IL23A             IL21              AMH             BMP7 
               0                0                0                0 
            BMP6             GDF6             BMP4             IL20 
               0                0                0                0 
            IL24             IL27            MTMR4              IL5 
               0                0                0                0 
            THPO              DHH            BMP8B          KREMEN1 
               0                0                0                0 
         KREMEN2             WNT2            WNT7A             IL31 
               0                0                0                0 
           BMP15            IFNL1            IFNL2            IFNL3 
               0                0                0                0 
            IL19             IL22            TGFB1             MST1 
               0                0                0                0 
            BDNF              PTN              MDK            GNAI2 
               0                0                0                0 
          PIK3R1          ALOX5AP            CCL19            CCL21 
               0                0                0                0 
           CCL25            INHBA            INHBB             INHA 
               0                0                0                0 
             ADM             ADM2            CALCA            CALCB 
               0                0                0                0 
            IAPP             APLN            APELA              NMB 
               0                0                0                0 
             GRP            BMP10             BMP5            BMP8A 
               0                0                0                0 
            GDF2             KNG1               C3               C5 
               0                0                0                0 
           RPS19              CCK             GAST            CCL27 
               0                0                0                0 
           CCL28            CCL11            CCL13            CCL15 
               0                0                0                0 
            CCL2            CCL24            CCL26             CCL7 
               0                0                0                0 
           CCL14            CCL16             CCL3           DEFB4B 
               0                0                0                0 
           CCL20             CCL1          RARRES2              CRH 
               0                0                0                0 
             UCN             UCN2             UCN3            YARS1 
               0                0                0                0 
           CXCL6            CXCL8            CXCL1            CXCL2 
               0                0                0                0 
           CXCL3            CXCL5             PPBP           CXCL12 
               0                0                0                0 
         

In [37]:
length(ligand_target_matrix["_ADD1_ADD2_EPB41_GYPC_SLC2A1_SLC4A1_STOM",])

[1] 1143

In [39]:
length(colnames(ligand_target_matrix))

[1] 1143

In [41]:
length(unique(lr_network_Omnipath$from))

[1] 1143

In [54]:
ligand_target_matrix["IL1R2",]["IL1A"]

IL1A 
   0

In [55]:
ligand_target_matrix["_EIF4A2_EIF4E_EIF4G1",]

CALM1          S100A10            WNT5A           CXCL16 
               0                0                0                0 
          CCL3L1             IL10           FLT3LG            CXCL9 
               0                0                0                0 
            CCL5             CCL8             CCL4             DKK1 
               0                0                0                0 
          CXCL10             TP53           TNFSF9          TNFSF10 
               0                0                0                0 
            FADD            GNRH1           CXCL11             IL13 
               0                0                0                0 
            BMP2             GDF5              NDP              IL3 
               0                0                0                0 
           IL12A              SHH              IHH             WNT1 
               0                0                0                0 
           IL23A             IL21              AMH             BMP7 
               0                0                0                0 
            BMP6             GDF6             BMP4             IL20 
               0                0                0                0 
            IL24             IL27            MTMR4              IL5 
               0                0                0                0 
            THPO              DHH            BMP8B          KREMEN1 
               0                0                0                0 
         KREMEN2             WNT2            WNT7A             IL31 
               0                0                0                0 
           BMP15            IFNL1            IFNL2            IFNL3 
               0                0                0                0 
            IL19             IL22            TGFB1             MST1 
               0                0                0                0 
            BDNF              PTN              MDK            GNAI2 
               0                0                0                0 
          PIK3R1          ALOX5AP            CCL19            CCL21 
               0                0                0                0 
           CCL25            INHBA            INHBB             INHA 
               0                0                0                0 
             ADM             ADM2            CALCA            CALCB 
               0                0                0                0 
            IAPP             APLN            APELA              NMB 
               0                0                0                0 
             GRP            BMP10             BMP5            BMP8A 
               0                0                0                0 
            GDF2             KNG1               C3               C5 
               0                0                0                0 
           RPS19              CCK             GAST            CCL27 
               0                0                0                0 
           CCL28            CCL11            CCL13            CCL15 
               0                0                0                0 
            CCL2            CCL24            CCL26             CCL7 
               0                0                0                0 
           CCL14            CCL16             CCL3           DEFB4B 
               0                0                0                0 
           CCL20             CCL1          RARRES2              CRH 
               0                0                0                0 
             UCN             UCN2             UCN3            YARS1 
               0                0                0                0 
           CXCL6            CXCL8            CXCL1            CXCL2 
               0                0                0                0 
           CXCL3            CXCL5             PPBP           CXCL12 
               0                0                0                0 
         

In [117]:
head(ligand_target_matrix)

,CALM1,S100A10,WNT5A,CXCL16,CCL3L1,IL10,FLT3LG,CXCL9,CCL5,CCL8,⋯,GFER,SEMA3E,TFF3,TTR,IL32,PYY2,CHRD,A2M,CSF3R,CIRBP
_ADD1_ADD2_EPB41_GYPC_SLC2A1_SLC4A1_STOM,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
_APH1A_NCSTN_PSEN1_PSENEN,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
_CCNC_CDK8_MED12_MED13_MED13L,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
_CHEBI:18420_DIS3L_EXOSC1_EXOSC10_EXOSC2_EXOSC3_EXOSC4_EXOSC5_EXOSC6_EXOSC7_EXOSC8_EXOSC9,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
_CRCP_POLR1C_POLR1D_POLR2E_POLR2F_POLR2H_POLR2K_POLR2L_POLR3A_POLR3B_POLR3C_POLR3D_POLR3E_POLR3F_POLR3G_POLR3H_POLR3K,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
_EIF4A2_EIF4E_EIF4G1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [112]:
"_ADD1_ADD2_EPB41_GYPC_SLC2A1_SLC4A1_STOM" %in% rownames(ligand_target_matrix)

[1] TRUE

In [116]:
"_ADD1_ADD2_EPB41_GYPC_SLC2A1_SLC4A1_STOM" %in% c(g)

[1] FALSE

In [119]:
weighted_networks

from,to,weight
<chr>,<chr>,<dbl>
_ADD1_ADD2_EPB41_GYPC_SLC2A1_SLC4A1_STOM,SPTB,0.025726594
_CCNC_CDK8_MED12_MED13_MED13L,MED1_MED10_MED11_MED14_MED15_MED16_MED17_MED18_MED19_MED20_MED21_MED22_MED23_MED24_MED25_MED26_MED27_MED28_MED29_MED30_MED31_MED4_MED6_MED7_MED8_MED9,0.030207910
_CHEBI:18420_DIS3L_EXOSC1_EXOSC10_EXOSC2_EXOSC3_EXOSC4_EXOSC5_EXOSC6_EXOSC7_EXOSC8_EXOSC9,FUS,0.017332671
_GRIN1_GRIN2A,CAMK2A,0.013437942
_GRIN1_GRIN2A,DLG4,0.012041060
_GRIN1_GRIN2B,CAMK2A,0.013437942
_GRIN1_GRIN2B,DLG4,0.012041060
_GRIN1_GRIN2C,CAMK2A,0.013437942
_GRIN1_GRIN2C,DLG4,0.012041060


In [134]:
"_ADD1_ADD2_EPB41_GYPC_SLC2A1_SLC4A1_STOM" %in% sig_network_Omnipath$from

[1] TRUE

In [136]:
filter(sig_network_Omnipath, from=="_ADD1_ADD2_EPB41_GYPC_SLC2A1_SLC4A1_STOM")

from,to,source,database
<chr>,<chr>,<chr>,<chr>
_ADD1_ADD2_EPB41_GYPC_SLC2A1_SLC4A1_STOM,SPTB,SIGNOR,SIGNOR


In [147]:
filter(sig_network_Omnipath, from==to)

from,to,source,database
<chr>,<chr>,<chr>,<chr>


In [153]:
print(ligand_target_matrix["_ADD1_ADD2_EPB41_GYPC_SLC2A1_SLC4A1_STOM",])

           CALM1          S100A10            WNT5A           CXCL16 
               0                0                0                0 
          CCL3L1             IL10           FLT3LG            CXCL9 
               0                0                0                0 
            CCL5             CCL8             CCL4             DKK1 
               0                0                0                0 
          CXCL10             TP53           TNFSF9          TNFSF10 
               0                0                0                0 
            FADD            GNRH1           CXCL11             IL13 
               0                0                0                0 
            BMP2             GDF5              NDP              IL3 
               0                0                0                0 
           IL12A              SHH              IHH             WNT1 
               0                0                0                0 
           IL23A             IL21 

In [161]:
"SPTB" %in% sig_network_Omnipath$to

[1] TRUE

In [174]:
print((ligand_target_matrix["SPTB",]))

           CALM1          S100A10            WNT5A           CXCL16 
    1.089946e-04     9.113793e-05     3.234523e-04     2.348328e-04 
          CCL3L1             IL10           FLT3LG            CXCL9 
    1.287386e-04     6.567735e-04     9.103512e-04     1.901992e-04 
            CCL5             CCL8             CCL4             DKK1 
    1.654997e-04     1.127494e-04     1.244757e-04     3.945657e-04 
          CXCL10             TP53           TNFSF9          TNFSF10 
    1.998870e-04     5.557627e-04     1.492832e-04     2.929035e-04 
            FADD            GNRH1           CXCL11             IL13 
    3.091995e-04     9.014043e-05     2.004069e-04     2.698499e-04 
            BMP2             GDF5              NDP              IL3 
    1.628401e-03     8.405751e-04     9.909294e-05     4.826474e-03 
           IL12A              SHH              IHH             WNT1 
    3.444644e-04     3.978687e-04     1.575215e-04     2.692896e-04 
           IL23A             IL21 

In [172]:
filter(sig_network_Omnipath, to=="SPTB")

from,to,source,database
<chr>,<chr>,<chr>,<chr>
ANK1_CD47_EPB42_GYPA_GYPB_ICAM4_RHAG_RHCE_SLC2A1_SLC4A1,SPTB,SIGNOR,SIGNOR
_ADD1_ADD2_EPB41_GYPC_SLC2A1_SLC4A1_STOM,SPTB,SIGNOR,SIGNOR


In [192]:
filter(lr_network_Omnipath, from=="CIRBP")

from,to,source,database
<chr>,<chr>,<chr>,<chr>
CIRBP,MAPK3,Wang,Wang


In [198]:
filter(weighted_networks$lr_sig, from=="CIRBP")

from,to,weight
<chr>,<chr>,<dbl>
CIRBP,MAPK3,0.06563177


In [209]:
filter(weighted_networks$lr_sig, from=="MAPK3")

from,to,weight
<chr>,<chr>,<dbl>
MAPK3,ABI1,3.05780629
MAPK3,ABLIM1,0.73101746
MAPK3,AKT1S1_DEPTOR_MLST8_MTOR_RPTOR,0.01546382
MAPK3,ALOX5,0.22696528
MAPK3,ALPP,0.12415078
MAPK3,AMPH,0.92313798
MAPK3,APBB1,1.13660517
MAPK3,APC_AXIN1_GSK3B,0.01384627
MAPK3,ARHGAP31,0.28841944


In [203]:
listofligands=filter(weighted_networks$lr_sig, from=="MAPK3")$to

In [218]:
"GATA1" %in% listofligands

[1] TRUE

In [216]:
filter(weighted_networks$gr, to=="SPTB")

from,to,weight
<chr>,<chr>,<dbl>
CTCF,SPTB,0.5658449
GATA1,SPTB,0.3621756
GATA2,SPTB,0.4798268
GATA6,SPTB,0.1141624
KLF1,SPTB,0.2917138
PKNOX1,SPTB,0.1141624
TAL1,SPTB,0.2917138
TEAD4,SPTB,0.2917138
TFAP2C,SPTB,0.4855442


In [237]:
"CTCF" %in% listofligands

[1] FALSE

In [223]:
filter(weighted_networks$gr, to=="SPTB")

from,to,weight
<chr>,<chr>,<dbl>
CTCF,SPTB,0.5658449
GATA1,SPTB,0.3621756
GATA2,SPTB,0.4798268
GATA6,SPTB,0.1141624
KLF1,SPTB,0.2917138
PKNOX1,SPTB,0.1141624
TAL1,SPTB,0.2917138
TEAD4,SPTB,0.2917138
TFAP2C,SPTB,0.4855442


In [ ]:
TAL1	SPTB	0.2917138
GATA6	SPTB	0.1141624
GATA1	SPTB	0.3621756

In [240]:
0.2917138*0.1141624*0.3621756*0.06563177

[1] 0.0007916138

In [241]:
2.955585e-03

[1] 0.002955585

In [243]:
0.1141624*0.3621756*0.06563177

[1] 0.002713666

In [254]:
0.2917138*0.3621756*0.06563177

[1] 0.01206144

In [249]:
0.1141624*0.3621756*0.06563177

[1] 0.002713666

In [255]:
head(ligand_target_matrix)

,CALM1,S100A10,WNT5A,CXCL16,CCL3L1,IL10,FLT3LG,CXCL9,CCL5,CCL8,⋯,GFER,SEMA3E,TFF3,TTR,IL32,PYY2,CHRD,A2M,CSF3R,CIRBP
_ADD1_ADD2_EPB41_GYPC_SLC2A1_SLC4A1_STOM,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
_APH1A_NCSTN_PSEN1_PSENEN,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
_CCNC_CDK8_MED12_MED13_MED13L,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
_CHEBI:18420_DIS3L_EXOSC1_EXOSC10_EXOSC2_EXOSC3_EXOSC4_EXOSC5_EXOSC6_EXOSC7_EXOSC8_EXOSC9,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
_CRCP_POLR1C_POLR1D_POLR2E_POLR2F_POLR2H_POLR2K_POLR2L_POLR3A_POLR3B_POLR3C_POLR3D_POLR3E_POLR3F_POLR3G_POLR3H_POLR3K,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
_EIF4A2_EIF4E_EIF4G1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [256]:
2.955585e-03

[1] 0.002955585

In [273]:
0.45727914*0.41091672*0.25442946*0.65644972*0.59763038*0.38658990+0.28732124

[1] 0.2945721

In [304]:
(3*optimized_parameters$damping_factor)*optimized_parameters$ltf_cutoff

[1] 1.781689

In [327]:
(0.45727914+0.41091672+0.25442946+0.65644972+0.59763038+0.38658990+0.28732124)/1.52

[1] 2.006985

In [271]:
optimized_parameters$source_weight_df

source,weight
<chr>,<dbl>
CellTalkDB,0.45727914
Fantom5_LRdb,0.41091672
HPRD,0.25442946
HPRD_LRdb,0.65644972
LRdb,0.59763038
TRIP,0.38658990
iTALK,0.28732124
Baccin2019,0.75803618
CellCall,0.14952847


In [307]:
filter(lr_network_Omnipath,from=="CALM1" & to=="TRPC3")

from,to,source,database
<chr>,<chr>,<chr>,<chr>
CALM1,TRPC3,CellTalkDB,CellTalkDB
CALM1,TRPC3,Fantom5_LRdb,Fantom5_LRdb
CALM1,TRPC3,HPRD,HPRD
CALM1,TRPC3,HPRD_LRdb,HPRD_LRdb
CALM1,TRPC3,LRdb,LRdb
CALM1,TRPC3,TRIP,TRIP
CALM1,TRPC3,iTALK,iTALK


In [345]:
sig_network_Omnipath$pair=paste(sig_network_Omnipath$from, sig_network_Omnipath$to, sep="_")
weighted_networks$lr_sig$pair=paste(weighted_networks$lr_sig$from, weighted_networks$lr_sig$to, sep="_")
lr_network_Omnipath$pair=paste(lr_network_Omnipath$from, lr_network_Omnipath$to, sep="_")

In [344]:
length(unique(weighted_networks$lr_sig$pair))

[1] 58820

In [346]:
length(unique(lr_network_Omnipath$pair))

[1] 7082

In [348]:
51738+7082

[1] 58820

In [343]:
length(unique(sig_network_Omnipath$pair))

[1] 51738

In [333]:
filter(weighted_networks$lr_sig, to=="TRPC3")

from,to,weight
<chr>,<chr>,<dbl>
ASPH,TRPC3,0.1931284
CALM1,TRPC3,1.5239942
CALM2,TRPC3,0.1931284
CALM3,TRPC3,0.1931284
CASR,TRPC3,0.1931284
EPOR,TRPC3,0.1931284
HOMER1,TRPC3,0.1931284
ITPR1,TRPC3,0.3202336
ITPR3,TRPC3,0.3202336


In [350]:
length(rownames(ligand_target_matrix))

[1] 13242

In [265]:
ligand_target_matrix["TRPC3",]

CALM1          S100A10            WNT5A           CXCL16 
    2.932618e-04     2.165692e-04     5.859553e-04     5.218872e-04 
          CCL3L1             IL10           FLT3LG            CXCL9 
    3.061173e-04     9.745173e-04     5.298195e-04     3.803819e-04 
            CCL5             CCL8             CCL4             DKK1 
    2.490185e-04     2.615611e-04     3.042103e-04     1.008030e-03 
          CXCL10             TP53           TNFSF9          TNFSF10 
    4.018300e-04     6.279289e-03     1.980335e-03     7.994466e-04 
            FADD            GNRH1           CXCL11             IL13 
    4.484102e-04     2.091953e-04     4.009116e-04     7.423979e-04 
            BMP2             GDF5              NDP              IL3 
    8.830023e-04     6.284095e-04     2.938087e-04     7.867718e-04 
           IL12A              SHH              IHH             WNT1 
    8.448800e-04     4.581447e-04     2.581945e-04     8.174115e-04 
           IL23A             IL21              AMH             BMP7 
    9.254463e-04     4.735453e-04     8.122399e-04     6.830377e-04 
            BMP6             GDF6             BMP4             IL20 
    6.445483e-04     5.746365e-04     8.765224e-04     7.293220e-04 
            IL24             IL27            MTMR4              IL5 
    7.017855e-04     1.390556e-03     1.178072e-02     1.211508e-03 
            THPO              DHH            BMP8B          KREMEN1 
    7.328928e-04     2.530037e-04     4.278023e-04     1.189558e-03 
         KREMEN2             WNT2            WNT7A             IL31 
    1.189558e-03     5.604078e-04     4.405951e-04     2.943623e-05 
           BMP15            IFNL1            IFNL2            IFNL3 
    9.157592e-04     5.283511e-04     5.127991e-04     4.807345e-04 
            IL19             IL22            TGFB1             MST1 
    5.439064e-04     9.085279e-04     2.233928e-03     1.314343e-03 
            BDNF              PTN              MDK            GNAI2 
    4.477905e-04     4.000137e-04     4.856252e-04     3.951701e-04 
          PIK3R1          ALOX5AP            CCL19            CCL21 
    1.047086e-03     0.000000e+00     3.141464e-04     3.481399e-04 
           CCL25            INHBA            INHBB             INHA 
    3.729335e-04     2.051937e-03     2.868057e-03     1.468041e-03 
             ADM             ADM2            CALCA            CALCB 
    1.984803e-04     4.584858e-05     3.543279e-05     4.116019e-05 
            IAPP             APLN            APELA              NMB 
    9.607770e-05     2.812568e-04     2.587149e-04     4.982984e-04 
             GRP            BMP10             BMP5            BMP8A 
    6.020599e-04     5.771174e-04     5.475947e-04     2.730387e-04 
            GDF2             KNG1               C3               C5 
    8.432559e-04     1.779290e-04     1.644164e-04     7.942145e-06 
           RPS19              CCK             GAST            CCL27 
    0.000000e+00     2.242181e-04     3.730707e-04     4.209377e-04 
           CCL28            CCL11            CCL13            CCL15 
    3.641005e-04     2.928246e-04     2.956054e-04     4.380836e-04 
            CCL2            CCL24            CCL26             CCL7 
    3.592134e-04     3.459490e-04     3.813490e-04     3.166352e-04 
           CCL14            CCL16             CCL3           DEFB4B 
    2.960024e-04     3.085603e-04     3.187147e-04     4.851115e-04 
           CCL20             CCL1          RARRES2              CRH 
    4.765866e-04     5.307119e-04     5.755634e-05     3.946646e-04 
             UCN             UCN2             UCN3            YARS1 
    5.258279e-04     4.296587e-04     4.189616e-04     6.666616e-04 
           CXCL6            CXCL8            CXCL1            CXCL2 
    5.354324e-04     4.567427e-04     5.158613e-04     5.280240e-04 
           CXCL3            CXCL5             PPBP           CXCL12 
    5.123539e-04     5.244436e-04     5.134606e-04     9.311788e-04 
         

In [308]:
head(lr_network_Omnipath)

from,to,source,database
<chr>,<chr>,<chr>,<chr>
CALM1,TRPC3,CellTalkDB,CellTalkDB
CALM1,TRPC3,Fantom5_LRdb,Fantom5_LRdb
CALM1,TRPC3,HPRD,HPRD
CALM1,TRPC3,HPRD_LRdb,HPRD_LRdb
CALM1,TRPC3,LRdb,LRdb
CALM1,TRPC3,TRIP,TRIP


In [316]:
matching_rows <- weighted_networks$lr_sig %>%
  filter(from %in% lr_network_Omnipath$from & to %in% lr_network_Omnipath$to)

In [319]:
str(lr_network_Omnipath)

tibble [26,804 × 4] (S3: tbl_df/tbl/data.frame)
 $ from    : chr [1:26804] "CALM1" "CALM1" "CALM1" "CALM1" ...
 $ to      : chr [1:26804] "TRPC3" "TRPC3" "TRPC3" "TRPC3" ...
 $ source  : chr [1:26804] "CellTalkDB" "Fantom5_LRdb" "HPRD" "HPRD_LRdb" ...
 $ database: chr [1:26804] "CellTalkDB" "Fantom5_LRdb" "HPRD" "HPRD_LRdb" ...


In [318]:
str(matching_rows)

tibble [7,874 × 3] (S3: tbl_df/tbl/data.frame)
 $ from  : chr [1:7874] "A2M" "ACE" "ACE2" "ACKR1" ...
 $ to    : chr [1:7874] "CHAT" "BDKRB2" "SLC6A19" "CCR5" ...
 $ weight: num [1:7874] 0.128 3.306 0.936 0.103 0.198 ...


In [314]:
filter(weighted_networks$lr_sig, to=="TRPC3")

from,to,weight
<chr>,<chr>,<dbl>
ASPH,TRPC3,0.1931284
CALM1,TRPC3,1.5239942
CALM2,TRPC3,0.1931284
CALM3,TRPC3,0.1931284
CASR,TRPC3,0.1931284
EPOR,TRPC3,0.1931284
HOMER1,TRPC3,0.1931284
ITPR1,TRPC3,0.3202336
ITPR3,TRPC3,0.3202336


In [312]:
ligand_target_matrix["TRPC3",]

CALM1          S100A10            WNT5A           CXCL16 
    2.932618e-04     2.165692e-04     5.859553e-04     5.218872e-04 
          CCL3L1             IL10           FLT3LG            CXCL9 
    3.061173e-04     9.745173e-04     5.298195e-04     3.803819e-04 
            CCL5             CCL8             CCL4             DKK1 
    2.490185e-04     2.615611e-04     3.042103e-04     1.008030e-03 
          CXCL10             TP53           TNFSF9          TNFSF10 
    4.018300e-04     6.279289e-03     1.980335e-03     7.994466e-04 
            FADD            GNRH1           CXCL11             IL13 
    4.484102e-04     2.091953e-04     4.009116e-04     7.423979e-04 
            BMP2             GDF5              NDP              IL3 
    8.830023e-04     6.284095e-04     2.938087e-04     7.867718e-04 
           IL12A              SHH              IHH             WNT1 
    8.448800e-04     4.581447e-04     2.581945e-04     8.174115e-04 
           IL23A             IL21              AMH             BMP7 
    9.254463e-04     4.735453e-04     8.122399e-04     6.830377e-04 
            BMP6             GDF6             BMP4             IL20 
    6.445483e-04     5.746365e-04     8.765224e-04     7.293220e-04 
            IL24             IL27            MTMR4              IL5 
    7.017855e-04     1.390556e-03     1.178072e-02     1.211508e-03 
            THPO              DHH            BMP8B          KREMEN1 
    7.328928e-04     2.530037e-04     4.278023e-04     1.189558e-03 
         KREMEN2             WNT2            WNT7A             IL31 
    1.189558e-03     5.604078e-04     4.405951e-04     2.943623e-05 
           BMP15            IFNL1            IFNL2            IFNL3 
    9.157592e-04     5.283511e-04     5.127991e-04     4.807345e-04 
            IL19             IL22            TGFB1             MST1 
    5.439064e-04     9.085279e-04     2.233928e-03     1.314343e-03 
            BDNF              PTN              MDK            GNAI2 
    4.477905e-04     4.000137e-04     4.856252e-04     3.951701e-04 
          PIK3R1          ALOX5AP            CCL19            CCL21 
    1.047086e-03     0.000000e+00     3.141464e-04     3.481399e-04 
           CCL25            INHBA            INHBB             INHA 
    3.729335e-04     2.051937e-03     2.868057e-03     1.468041e-03 
             ADM             ADM2            CALCA            CALCB 
    1.984803e-04     4.584858e-05     3.543279e-05     4.116019e-05 
            IAPP             APLN            APELA              NMB 
    9.607770e-05     2.812568e-04     2.587149e-04     4.982984e-04 
             GRP            BMP10             BMP5            BMP8A 
    6.020599e-04     5.771174e-04     5.475947e-04     2.730387e-04 
            GDF2             KNG1               C3               C5 
    8.432559e-04     1.779290e-04     1.644164e-04     7.942145e-06 
           RPS19              CCK             GAST            CCL27 
    0.000000e+00     2.242181e-04     3.730707e-04     4.209377e-04 
           CCL28            CCL11            CCL13            CCL15 
    3.641005e-04     2.928246e-04     2.956054e-04     4.380836e-04 
            CCL2            CCL24            CCL26             CCL7 
    3.592134e-04     3.459490e-04     3.813490e-04     3.166352e-04 
           CCL14            CCL16             CCL3           DEFB4B 
    2.960024e-04     3.085603e-04     3.187147e-04     4.851115e-04 
           CCL20             CCL1          RARRES2              CRH 
    4.765866e-04     5.307119e-04     5.755634e-05     3.946646e-04 
             UCN             UCN2             UCN3            YARS1 
    5.258279e-04     4.296587e-04     4.189616e-04     6.666616e-04 
           CXCL6            CXCL8            CXCL1            CXCL2 
    5.354324e-04     4.567427e-04     5.158613e-04     5.280240e-04 
           CXCL3            CXCL5             PPBP           CXCL12 
    5.123539e-04     5.244436e-04     5.134606e-04     9.311788e-04 
         

In [310]:
filter(sig_network_Omnipath, to=="TRPC3")

from,to,source,database
<chr>,<chr>,<chr>,<chr>
TRPC1,TRPC3,BioGRID,BioGRID
TRPC1,TRPC3,HPRD,HPRD
TRPC1,TRPC3,TRIP,TRIP
CASR,TRPC3,TRIP,TRIP
HOMER1,TRPC3,TRIP,TRIP
ITPR1,TRPC3,HPRD,HPRD
ITPR1,TRPC3,TRIP,TRIP
ITPR3,TRPC3,HPRD,HPRD
ITPR3,TRPC3,TRIP,TRIP
